In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predict-potential-spammers-on-fiverr/sample_submission.csv
/kaggle/input/predict-potential-spammers-on-fiverr/train.csv
/kaggle/input/predict-potential-spammers-on-fiverr/test.csv


In [2]:
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost as xgb


In [3]:
train = pd.read_csv('/kaggle/input/predict-potential-spammers-on-fiverr/train.csv')
test = pd.read_csv('/kaggle/input/predict-potential-spammers-on-fiverr/test.csv')
train.label.value_counts()

0    446477
1     12321
Name: label, dtype: int64

In [4]:
print(f"Dimensions of train data : {train.shape}")
print(f"Dimensions of test data : {test.shape}")

Dimensions of train data : (458798, 53)
Dimensions of test data : (24148, 52)


In [5]:
features = [feature for feature in train if feature.startswith('X')]
# print(features)
for feat in features:
    if (train[feat].nunique() == 1):
        print(feat)

X27
X29
X30
X33
X46
X47
X48


In [6]:
constant_columns = list()
for feat in features:
    if (train[feat].nunique() == 1):
        constant_columns.append(feat)
train.drop(columns = constant_columns, inplace = True)
train.shape

(458798, 46)

In [7]:
train['X13'].fillna(value = round(train['X13'].mean()), inplace = True)
train.X13.isna().sum()


0

In [8]:
test.drop(columns = constant_columns, inplace=True)
train.shape, test.shape

((458798, 46), (24148, 45))

In [9]:
train = train.drop(columns = ['user_id'])
test = test.drop(columns = ['user_id'])
train.head()

,label,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X39,X40,X41,X42,X43,X44,X45,X49,X50,X51
0,0,20972,14,13,3,11,1,2,15,26,...,0,0,0,0,0,0,0,0,1,0
1,0,7362,213,71,3,11,1,2,15,26,...,0,0,0,0,0,0,0,0,0,0
2,0,21216,215,71,3,11,1,2,15,14,...,0,0,0,0,0,0,0,0,0,0
3,0,2261,212,71,2,8,1,2,15,14,...,0,0,0,0,0,0,0,1,0,0
4,0,4543,213,71,2,8,1,2,15,14,...,0,0,0,0,0,0,0,0,0,0


In [10]:
X = train.drop(columns = ['label'])
y = train[['label']]


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.3)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape


((321158, 44), (137640, 44), (321158, 1), (137640, 1))

In [12]:
model_xgb = xgb.XGBClassifier(n_estimators = 700)


In [13]:
model_xgb.fit(X_train, y_train, 
            early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)
print(model_xgb)

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=700,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)


In [14]:
model_xgb_tuned = xgb.XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              monotone_constraints='()', n_estimators=700,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1)


# model = xgb.XGBClassifier(max_depth = 8, learning_rate=0.22,n_estimators=250,subsample = 0.95,reg_alpha = 0.3, reg_lambda = 0.95, gamma = 0.1)

# XGBClassifier(max_depth = 8,
#                                 learning_rate=0.22,
#                                 n_estimators=250,
#                                 reg_alpha = 0.35,
#                                 reg_lambda = 0.95,
#                                 gamma = 0.1))

In [15]:
model_xgb_tuned.fit(X, y,
             verbose=False)
print(model_xgb)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=700,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)


In [16]:
predictions_train = model_xgb_tuned.predict(X)
print(metrics.classification_report(y, predictions_train))
print(metrics.confusion_matrix(y, predictions_train))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    446477
           1       0.96      0.84      0.90     12321

    accuracy                           0.99    458798
   macro avg       0.98      0.92      0.95    458798
weighted avg       0.99      0.99      0.99    458798

[[446039    438]
 [  1971  10350]]


In [17]:
submission = pd.read_csv('/kaggle/input/predict-potential-spammers-on-fiverr/sample_submission.csv')
submission.head()

,user_id,label
0,57,0
1,58,0
2,59,0
3,60,0
4,61,0


In [18]:
predictions = model_xgb_tuned.predict(test)
predictions

array([0, 0, 0, ..., 0, 0, 0])

In [19]:
submission.label = predictions
submission.columns = ['user_id', 'prediction']
submission.prediction.value_counts()

0    23176
1      972
Name: prediction, dtype: int64

In [20]:
submission.to_csv('submission.csv', index = False)

In [21]:
# model.fit(X, y)
# print(model)

In [22]:
# predictions = model.predict(test)
# np.unique(predictions, return_counts=True)

In [23]:
# predictions_train = model.predict(X)
# print(metrics.classification_report(y, predictions_train))
# print(metrics.confusion_matrix(y, predictions_train))

In [24]:
# submission = pd.read_csv('/kaggle/input/predict-potential-spammers-on-fiverr/sample_submission.csv')
# submission.head()

In [25]:
# submission.label = predictions
# submission.columns = ['user_id', 'prediction']

In [26]:
# submission.to_csv('submission.csv', index = False)